In [2]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
import requests

In [3]:
pi = 0
pf = 13
main_url = "https://investigadores.unison.mx/es/persons/?format=&page="
numbers = list(range(pi,pf+1))
numbers

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [4]:
pages = []
for n in numbers:
    pages.append(main_url+str(n))
pages

['https://investigadores.unison.mx/es/persons/?format=&page=0',
 'https://investigadores.unison.mx/es/persons/?format=&page=1',
 'https://investigadores.unison.mx/es/persons/?format=&page=2',
 'https://investigadores.unison.mx/es/persons/?format=&page=3',
 'https://investigadores.unison.mx/es/persons/?format=&page=4',
 'https://investigadores.unison.mx/es/persons/?format=&page=5',
 'https://investigadores.unison.mx/es/persons/?format=&page=6',
 'https://investigadores.unison.mx/es/persons/?format=&page=7',
 'https://investigadores.unison.mx/es/persons/?format=&page=8',
 'https://investigadores.unison.mx/es/persons/?format=&page=9',
 'https://investigadores.unison.mx/es/persons/?format=&page=10',
 'https://investigadores.unison.mx/es/persons/?format=&page=11',
 'https://investigadores.unison.mx/es/persons/?format=&page=12',
 'https://investigadores.unison.mx/es/persons/?format=&page=13']

In [5]:
pages[0]

'https://investigadores.unison.mx/es/persons/?format=&page=0'

In [6]:
links = []
for page in pages:
    url = page
    url_get = requests.get(url)
    html = BeautifulSoup(url_get.content, "html.parser")
    for i in html.find_all("h3", class_="title"):
        links.append(i.a["href"])

In [7]:
len(links)

608

In [8]:
df = pd.DataFrame(columns=["nombre", "correo", "publicaciones", "citas", "indice h", "link"])
df

,nombre,correo,publicaciones,citas,indice h,link


In [9]:
for link in links:
    url = link +"/publications/"
    url_get = requests.get(url)
    html = BeautifulSoup(url_get.content, 'html.parser')

    nombre = html.find("h1").text

    try:
        correo = html.find("li", class_="emails").a["href"]
        correo = correo.replace("mailto:","")
    except:
        correo = np.nan

    try:
        citas = html.find("li", class_="citations").span.span.text
    except:
        citas = np.nan
        
    try:
        h = html.find("li", class_="h-index").span.span.text
    except:
        h = np.nan
        
    try:
        publicaciones = html.find("a",class_="portal_link count increment-counter").span.text
        publicaciones = publicaciones.replace("\n","")
        publicaciones = publicaciones.replace(" ","")
    except:
        publicaciones = np.nan
        
    person = pd.DataFrame(columns=df.columns)
    person.loc[0] = [nombre, correo, publicaciones, citas, h, link]
    person
        
    df = df.append(person)

df.reset_index(drop=True, inplace=True)

In [10]:
df

,nombre,correo,publicaciones,citas,indice h,link
0,Raúl Aceves Torres,raul.aceves@unison.mx,19,143,8,https://investigadores.unison.mx/es/persons/ra...
1,Milka del Carmen Acosta Enríquez,milka.acosta@unison.mx,25,44,4,https://investigadores.unison.mx/es/persons/mi...
2,Andrés Acosta Félix,andres.acosta@unison.mx,NaN,NaN,NaN,https://investigadores.unison.mx/es/persons/an...
3,Heriberto Acuña Campa,heriberto.acuna@unison.mx,6,105,5,https://investigadores.unison.mx/es/persons/he...
4,Karla Fabiola Acuña Meléndrez,karla.acuna@unison.mx,NaN,NaN,NaN,https://investigadores.unison.mx/es/persons/ka...
...,...,...,...,...,...,...
603,Andrea Guadalupe Zavala Andrea,andrea.zavala@unison.mx,5,55,4,https://investigadores.unison.mx/es/persons/an...
604,Paul Zavala Rivera,paul.zavala@unison.mx,26,242,8,https://investigadores.unison.mx/es/persons/pa...
605,Federico Zayas Perez,federico.zayas@unison.mx,3,1,1,https://investigadores.unison.mx/es/persons/fe...
606,María Elena Zayas Saucedo,mzayas@difus.uson.mx,23,205,8,https://investigadores.unison.mx/es/persons/ma...


In [16]:
df.sort_values(by="citas",ascending=False)

,nombre,correo,publicaciones,citas,indice h,link
76,Armando Burgos Hernández,armando.burgos@unison.mx,85,991,15,https://investigadores.unison.mx/es/persons/ar...
528,Juan Pablo Soto Barrera,juanpablo.soto@unison.mx,41,99,5,https://investigadores.unison.mx/es/persons/ju...
127,Mario Cortez Rocha,mario.cortez@unison.mx,69,955,15,https://investigadores.unison.mx/es/persons/ma...
580,Carlos Arturo Velázquez Contreras,velaz@guayacan.uson.mx,64,950,17,https://investigadores.unison.mx/es/persons/ca...
279,Irlanda Lagarda Diaz,irlanda.lagarda@unison.mx,5,94,4,https://investigadores.unison.mx/es/persons/ir...
...,...,...,...,...,...,...
577,Ramón Arturo Vega Robles,arturo.vega@unison.mx,NaN,NaN,NaN,https://investigadores.unison.mx/es/persons/ra...
586,Joel Alfonso Verdugo Cordova,joel.verdugo@unison.mx,NaN,NaN,NaN,https://investigadores.unison.mx/es/persons/jo...
587,María Leticia Verdugo Tapia,marialeticia.verdugo@unison.mx,NaN,NaN,NaN,https://investigadores.unison.mx/es/persons/ma...
602,Lidia Amalia Zallas Esquer,lidia.zallas@unison.mx,NaN,NaN,NaN,https://investigadores.unison.mx/es/persons/li...
